In [ ]:
pip install wandb --upgrade

In [ ]:
!pip install accelerate

In [ ]:
!pip install sacremoses

In [ ]:
!pip install evaluate

In [ ]:
outputName = "masrawy-english-arabic-translator-opus-big-v1"

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoModel

modelName = "Helsinki-NLP/opus-mt-tc-big-en-ar"
tokenizer = AutoTokenizer.from_pretrained(modelName)
model = AutoModelForSeq2SeqLM.from_pretrained(modelName)

In [ ]:
from datasets import load_dataset
dataset = load_dataset("dataset") ##columns should include arabic, english
dataset = dataset.shuffle(seed=50)
print(dataset['train'][0], dataset['train'][5])
dataset = dataset["train"].train_test_split(test_size=0.05)

In [ ]:
source_lang = "english"
target_lang = "arabic"
prefix = "" ## if needed

def preprocess_function(examples):
    inputs = [prefix + example for example in examples[source_lang]]
    targets = [example for example in examples[target_lang]]
   
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, add_special_tokens=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=512, truncation=True, add_special_tokens=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs
tokenized_dataset = dataset.map(preprocess_function, batched=True)

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./"+ outputName,
    evaluation_strategy="steps",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    save_total_limit=3,
    num_train_epochs=10,
    eval_steps=1000,
    save_steps=100,
    gradient_checkpointing=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# To continue training from checkpoint
# trainer.train(resume_from_checkpoint=True)

# To start training fresh
trainer.train()


In [ ]:
trainer.push_to_hub("oddadmix/"+outputName)